## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [37]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hambantota,LK,2021-06-03 22:05:45,6.1241,81.1185,79.14,100,40,15.52,scattered clouds
1,1,Punta Arenas,CL,2021-06-03 22:05:46,-53.1500,-70.9167,35.11,93,100,11.99,light rain
2,2,Port Keats,AU,2021-06-03 22:05:48,-14.2500,129.5500,74.53,64,92,7.61,overcast clouds
3,3,Nikolskoye,RU,2021-06-03 22:05:50,59.7035,30.7861,57.09,82,20,4.72,few clouds
4,4,Vardo,NO,2021-06-03 22:05:51,70.3705,31.1107,51.03,63,20,9.95,few clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input("What is the minimum temperature you would like for your trip?"))
max_temp = int(input("What is the maximum temperature you would like for your trip?"))

In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
filtered_city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Nikolskoye,RU,2021-06-03 22:05:50,59.7035,30.7861,57.09,82,20,4.72,few clouds
4,4,Vardo,NO,2021-06-03 22:05:51,70.3705,31.1107,51.03,63,20,9.95,few clouds
6,6,Chokurdakh,RU,2021-06-03 22:05:54,70.6333,147.9167,57.34,58,100,2.46,overcast clouds
18,18,Chibombo,ZM,2021-06-03 22:06:12,-14.6569,28.0706,57.74,55,12,3.62,few clouds
19,19,Bilibino,RU,2021-06-03 22:06:13,68.0546,166.4372,57.78,34,12,1.70,few clouds


In [40]:
# 4a. Determine if there are any empty rows.
filtered_city_data_df.isna()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False
18,False,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
669,False,False,False,False,False,False,False,False,False,False,False
675,False,False,False,False,False,False,False,False,False,False,False
676,False,False,False,False,False,False,False,False,False,False,False
677,False,False,False,False,False,False,False,False,False,False,False


In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df = filtered_city_data_df.dropna()
filtered_city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Nikolskoye,RU,2021-06-03 22:05:50,59.7035,30.7861,57.09,82,20,4.72,few clouds
4,4,Vardo,NO,2021-06-03 22:05:51,70.3705,31.1107,51.03,63,20,9.95,few clouds
6,6,Chokurdakh,RU,2021-06-03 22:05:54,70.6333,147.9167,57.34,58,100,2.46,overcast clouds
18,18,Chibombo,ZM,2021-06-03 22:06:12,-14.6569,28.0706,57.74,55,12,3.62,few clouds
19,19,Bilibino,RU,2021-06-03 22:06:13,68.0546,166.4372,57.78,34,12,1.70,few clouds


In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,
4,Vardo,NO,51.03,few clouds,70.3705,31.1107,
6,Chokurdakh,RU,57.34,overcast clouds,70.6333,147.9167,
18,Chibombo,ZM,57.74,few clouds,-14.6569,28.0706,
19,Bilibino,RU,57.78,few clouds,68.0546,166.4372,
22,Port Alfred,ZA,56.59,broken clouds,-33.5906,26.8910,
27,Bredasdorp,ZA,55.27,overcast clouds,-34.5322,20.0403,
28,Vagur,FO,48.97,scattered clouds,61.4732,-6.8119,
29,Yellowknife,CA,51.84,light rain,62.4560,-114.3525,
32,New Norfolk,AU,47.70,overcast clouds,-42.7826,147.0587,


In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [44]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,Tourist House - Sablino
4,Vardo,NO,51.03,few clouds,70.3705,31.1107,Vardø Hotel AS
6,Chokurdakh,RU,57.34,overcast clouds,70.6333,147.9167,
18,Chibombo,ZM,57.74,few clouds,-14.6569,28.0706,Chibombo District Council Guest House
19,Bilibino,RU,57.78,few clouds,68.0546,166.4372,Gostinitsa Severyanka
...,...,...,...,...,...,...,...
669,Wufeng,CN,51.21,overcast clouds,27.4500,104.8333,中山宾馆
675,Potosi,BO,46.76,scattered clouds,-19.5836,-65.7531,Hostal La Casona Potosí
676,Helong,CN,55.67,scattered clouds,42.5397,128.9972,Helong Hotel
677,Jiuquan,CN,59.70,clear sky,39.7432,98.5174,Jiuquan Hotel


In [45]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nikolskoye,RU,57.09,few clouds,59.7035,30.7861,Tourist House - Sablino
4,Vardo,NO,51.03,few clouds,70.3705,31.1107,Vardø Hotel AS
18,Chibombo,ZM,57.74,few clouds,-14.6569,28.0706,Chibombo District Council Guest House
19,Bilibino,RU,57.78,few clouds,68.0546,166.4372,Gostinitsa Severyanka
22,Port Alfred,ZA,56.59,broken clouds,-33.5906,26.8910,The Halyards Hotel
...,...,...,...,...,...,...,...
666,Ilinskiy,RU,58.51,clear sky,55.6190,38.1182,Around the world
669,Wufeng,CN,51.21,overcast clouds,27.4500,104.8333,中山宾馆
675,Potosi,BO,46.76,scattered clouds,-19.5836,-65.7531,Hostal La Casona Potosí
676,Helong,CN,55.67,scattered clouds,42.5397,128.9972,Helong Hotel


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Search_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))